### get_schedule

In [1]:
import threading
import time

In [2]:
import pandas as pd
import numpy as np
import re
import os
import datetime as dt
from bs4 import BeautifulSoup, Comment
import requests

###############
#  NBA Class  #
###############

class NBA:
    """
    NBA - Documentation goes here
    """
    
    def __init__(self, url, table_id, column_schema, row_schema, data_schema,
                 use_links=[], add_links={}, name_change={}, filter_rows = {}, ):
        self.url = url
        self.table_id = table_id
        self.use_links = use_links
        self.add_links = add_links
        self.column_schema = column_schema
        self.row_schema = row_schema
        self.data_schema = data_schema
        self.name_change = name_change
        self.filter_rows = filter_rows
        
        try:
            self.get_soup()
            self.get_columns()
            self.get_rows()               
            self.get_data()
            self.add_link()
            self.filter_by_value()
        except:
            self.df = pd.DataFrame()

    def get_soup(self):

        r = requests.get(self.url)
        soup = BeautifulSoup(r.content, 'lxml')
        self.soup = soup.find('table', {"id": self.table_id})
        
    def get_columns(self):

        base="self.soup"
        add=".findAll('{}'){}"

        for i in range(len(self.column_schema['column_attrs'])):
            if self.column_schema['offset'][i] != None:
                base = f"{base}{add.format(self.column_schema['column_attrs'][i],'['+str(self.column_schema['offset'][i])+']')}"
            else:
                base = f"{base}{add.format(self.column_schema['column_attrs'][i],'')}"

        loop = f"[x.getText() for x in {base}][{self.column_schema['shift']}:]"

        self.columns = eval(loop)

        for k, v in self.name_change.items():
            self.columns[k] = v   
            
    def get_rows(self):

        base="self.soup"
        add=".findAll('{}'){}"

        for i in range(len(self.row_schema['row_attrs'])):
            if self.row_schema['row_offset'][i] != None:
                base = f"{base}{add.format(self.row_schema['row_attrs'][i], '['+str(self.row_schema['row_offset'][i])+']')}"
            else:
                base = f"{base}{add.format(self.row_schema['row_attrs'][i], '')}"

        self.rows = eval(base)

    def get_data(self):

        data = [[x.getText() if j not in self.use_links else x.a['href'] \
                 for j, x in enumerate(self.rows[i].findAll(self.data_schema['data_attrs']))][self.data_schema["data_offset"]:] \
                for i in range(len(self.rows))]

        max_list_len = max(list(map(lambda x: len(x), data)))
        self.data = [self.pad_list(sub,max_list_len) for sub in data if len(sub) > 0]

        self.df = pd.DataFrame(data=data, columns=self.columns)
        
    def add_link(self):

        for key, value in self.add_links.items():
            self.df[key] = [[x.a['href'] for j,x in enumerate(self.rows[i].findAll(self.data_schema['data_attrs']))                              if j == value][0] for i in range(len(self.data))]  
            
    def pad_list(self, l,n):
        while len(l) < n:
            l.append("")
        return l
    
    def filter_by_value(self):
        for key, value in self.filter_rows.items():
            self.df = self.df[self.df[key] != value].reset_index(drop=True)

###################
#  End of script  #
###################

In [3]:
def get_schedule(year, month):
    
    sch_url = f"https://www.basketball-reference.com/leagues/NBA_{year}_games-{month}.html"

    sch_col_attrs = {"column_attrs": ['thead', 'th'], "shift": 0, "offset": ['0', None]}
    sch_row_attrs = {"row_attrs": ['tbody', 'tr'], "row_offset": ['0', None]}
    sch_data_attrs = {"data_attrs": ['th', 'td'], "data_offset": 0}
    
    sch = NBA(url = sch_url, 
              table_id = 'schedule',
              column_schema = sch_col_attrs, 
              row_schema = sch_row_attrs, 
              data_schema = sch_data_attrs, 
              name_change = {3: "PTS_Vis",5: "PTS_Home", 6:"Boxscore", 7: "Overtime"},
              use_links=[6],
              filter_rows = {"Date": "Playoffs"})   

    sch.df['season'] = year
    
    if len(sch.df) > 0:
        sch.df.to_csv(f"data/schedule/{year}_{month}.csv", index=False)

In [4]:
def collapse_files(folder, save):
    
    final = pd.DataFrame()
    
    for file in os.listdir(folder):
        if file == ".ipynb_checkpoints":
            continue
        final = pd.concat([final, pd.read_csv(f"{folder}/{file}")])
        os.remove(f"{folder}/{file}")
    
    final.to_csv(f"{folder}/{save}", index=False)

In [5]:
if __name__ == '__main__':
    
    if not os.path.exists("data/schedule"):
        os.makedirs("data/schedule")
        
    years = list(range(2001, 2023))
    
    months = ["october", "november", "december", "january", "february", "march", "april", "may", "june"]
    
    months_2020 = ["october-2019", "november", "december", "january", "february", "march", "july",
                   "august", "september", "october-2020"]
    
    months_2021 = ["december", "january", "february", "march", "april", "may", "june", "july"]
    
    for year in years:
        print(f"Getting {year} NBA data.")
        
        if year == 2021:
            loop_months = months_2021
        elif year == 2020:
            loop_monts = months_2020
        else:
            loop_months = months
        
        for month in months:
            clean_month = month.replace('-', '')
            exec(f"t_{clean_month} = threading.Thread(target=get_schedule, args=(year, month,))")
            exec(f"t_{clean_month}.start()")
        for month in months:
            exec(f"t_{clean_month}.join()")

Getting 2001 NBA data.
Getting 2002 NBA data.
Getting 2003 NBA data.
Getting 2004 NBA data.
Getting 2005 NBA data.
Getting 2006 NBA data.
Getting 2007 NBA data.
Getting 2008 NBA data.
Getting 2009 NBA data.
Getting 2010 NBA data.
Getting 2011 NBA data.
Getting 2012 NBA data.
Getting 2013 NBA data.
Getting 2014 NBA data.
Getting 2015 NBA data.
Getting 2016 NBA data.
Getting 2017 NBA data.
Getting 2018 NBA data.
Getting 2019 NBA data.
Getting 2020 NBA data.
Getting 2021 NBA data.
Getting 2022 NBA data.


In [6]:
collapse_files("data/schedule", "nba_schedule.csv")

In [7]:
os.listdir('data/schedule')

['nba_schedule.csv', '.ipynb_checkpoints']

### test

In [8]:
import pandas as pd

In [9]:
df = pd.read_csv("data/schedule/nba_schedule.csv")

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27920 entries, 0 to 27919
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Date             27920 non-null  object
 1   Start (ET)       27920 non-null  object
 2   Visitor/Neutral  27920 non-null  object
 3   PTS_Vis          27920 non-null  int64 
 4   Home/Neutral     27920 non-null  object
 5   PTS_Home         27920 non-null  int64 
 6   Boxscore         27920 non-null  object
 7   Overtime         1694 non-null   object
 8   Attend.          27905 non-null  object
 9   Arena            27920 non-null  object
 10  Notes            49 non-null     object
 11  season           27920 non-null  int64 
dtypes: int64(3), object(9)
memory usage: 2.6+ MB


In [11]:
df.season.value_counts()

2022    1323
2014    1319
2006    1319
2008    1316
2016    1316
2009    1315
2005    1314
2013    1314
2018    1312
2019    1312
2010    1312
2015    1311
2011    1311
2007    1309
2017    1309
2003    1277
2004    1271
2001    1260
2002    1260
2021    1163
2012    1074
2020     903
Name: season, dtype: int64

In [12]:
df.columns = ['Date', 'Start', 'Visitor', 'PTS_Vis', 'Home',
       'PTS_Home', 'Boxscore', 'Overtime', 'Attendance', 'Arena', 'Notes',
       'season']

In [13]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()


In [18]:
job_config = bigquery.LoadJobConfig(
    write_disposition = 'WRITE_TRUNCATE'
)

In [19]:
table_ref = "dulcet-name-296415.nba_test.schedule"

job = client.load_table_from_dataframe(df, table_ref, job_config=job_config, location="US")

job.result()  # Waits for table load to complete.

LoadJob<project=dulcet-name-296415, location=US, id=4c9292a8-680c-4e1c-b04a-c01928157292>